# Pricing Logic Scheduler

Runs pricing notebooks based on scheduled times (Cairo timezone).

## Schedule
| Time | Modules |
|------|--------|
| 8 AM | Data Extraction + Module 2 (Initial Price Push) |
| 9 AM | Module 4 (Hourly Updates) + Treasure Hunt |
| 12 PM | Module 3 (Periodic Actions) |
| 1 PM | Module 4 (Hourly Updates) |
| 2 PM | Module 4 (Hourly Updates) |
| 3 PM | Module 3 (Periodic Actions) |
| 4 PM | Module 4 (Hourly Updates) |
| 5 PM | Module 4 (Hourly Updates) |
| 6 PM | Module 3 (Periodic Actions) |
| 7 PM | Module 4 (Hourly Updates) |
| 8 PM | Module 4 (Hourly Updates) |
| 9 PM | Module 3 (Periodic Actions) |
| 10 PM | Module 4 (Hourly Updates) |
| 11 PM | Module 3 (Periodic Actions) |

## Usage
- Run the **"Run Continuous Scheduler"** cell to start the scheduler loop
- Run the **"Run Once"** cell to execute only the current hour's tasks
- Run the **"Test Mode"** cell to see what would run without executing


In [1]:
# =============================================================================
# IMPORTS & CONFIGURATION
# =============================================================================
import os
import sys
import time
import subprocess
from datetime import datetime
import pytz

# Import Slack notification function
sys.path.insert(0, os.path.dirname(os.path.abspath('__file__')) or os.getcwd())
from common_functions import send_text_slack

# Slack channel for scheduler alerts
SLACK_CHANNEL = 'new-pricing-logic'

# Cairo timezone
CAIRO_TZ = pytz.timezone('Africa/Cairo')

# Base directory (where this notebook is located)
BASE_DIR = os.path.dirname(os.path.abspath('__file__'))
if BASE_DIR == '':
    BASE_DIR = os.getcwd()
MODULES_DIR = os.path.join(BASE_DIR, 'modules')

# Notebook paths
NOTEBOOKS = {
    'data_extraction': os.path.join(BASE_DIR, 'data_extraction.ipynb'),
    'wholeSale': os.path.join(BASE_DIR, 'whole_sale_new_logic.ipynb'),
    'module_2': os.path.join(MODULES_DIR, 'module_2_initial_price_push.ipynb'),
    'module_3': os.path.join(MODULES_DIR, 'module_3_periodic_actions.ipynb'),
    'module_4': os.path.join(MODULES_DIR, 'module_4_hourly_updates.ipynb'),
    'treasure_hunt': os.path.join(BASE_DIR, 'treasure_hunt_scheduler.ipynb'),
}

# Schedule configuration (hours in 24h format)
SCHEDULE = {                         
    8:  ['data_extraction', 'module_2'],  
    9:  ['module_4', 'treasure_hunt'],    # 9 AM - Added Treasure Hunt
    10: ['module_4'],
    11: ['module_4'],
    12: ['module_3'],                          # 12 PM
    13: ['module_4','wholeSale'],              # 1 PM
    14: ['module_4'],                          # 2 PM
    15: ['module_4'],                          # 3 PM
    16: ['module_4'],                          # 4 PM
    17: ['module_3'],                          # 5 PM
    18: ['module_4','wholeSale'],              # 6 PM
    19: ['module_4'],                          # 7 PM
    20: ['module_4'],                          # 8 PM
    21: ['module_4'],                          # 9 PM
    22: ['module_4'],                          # 10 PM
    23: ['module_3'],                          # 11 PM
}

# Log file
LOG_FILE = os.path.join(BASE_DIR, 'scheduler.log')

print(f"✓ Configuration loaded")
print(f"  Base directory: {BASE_DIR}")
print(f"  Modules directory: {MODULES_DIR}")
print(f"  Log file: {LOG_FILE}")
print(f"  Current Cairo time: {datetime.now(CAIRO_TZ).strftime('%Y-%m-%d %H:%M:%S')}")


✓ Configuration loaded
  Base directory: /home/ec2-user/SageMaker/Pricing Runs/Prediction_Scripts_2/Happy_hour/git/Mustafa/Pricing Logic
  Modules directory: /home/ec2-user/SageMaker/Pricing Runs/Prediction_Scripts_2/Happy_hour/git/Mustafa/Pricing Logic/modules
  Log file: /home/ec2-user/SageMaker/Pricing Runs/Prediction_Scripts_2/Happy_hour/git/Mustafa/Pricing Logic/scheduler.log
  Current Cairo time: 2026-02-10 11:01:20


In [2]:
# =============================================================================
# HELPER FUNCTIONS
# =============================================================================

def get_cairo_time():
    """Get current time in Cairo timezone."""
    return datetime.now(CAIRO_TZ)


def log_message(message, also_print=True):
    """Log a message to file and optionally print to console."""
    timestamp = get_cairo_time().strftime('%Y-%m-%d %H:%M:%S')
    log_line = f"[{timestamp}] {message}"
    
    if also_print:
        print(log_line)
    
    try:
        with open(LOG_FILE, 'a', encoding='utf-8') as f:
            f.write(log_line + '\n')
    except Exception as e:
        print(f"Warning: Could not write to log file: {e}")


def get_scheduled_notebooks(hour):
    """Get list of notebooks scheduled for a given hour."""
    return SCHEDULE.get(hour, [])


def print_schedule():
    """Print the full schedule."""
    print("\n" + "="*60)
    print("PRICING LOGIC SCHEDULER - SCHEDULE")
    print("="*60)
    print(f"{'Hour':<10} {'Notebooks':<50}")
    print("-"*60)
    
    for hour in sorted(SCHEDULE.keys()):
        notebooks = SCHEDULE[hour]
        hour_str = f"{hour:02d}:00"
        notebooks_str = ', '.join(notebooks)
        print(f"{hour_str:<10} {notebooks_str:<50}")
    
    print("="*60)
    print(f"\nAll times are in Cairo timezone ({CAIRO_TZ})")
    print(f"Current Cairo time: {get_cairo_time().strftime('%Y-%m-%d %H:%M:%S')}")
    print("="*60 + "\n")


def send_slack_alert(notebook_name, error_message, hour):
    """
    Send a Slack alert when a notebook fails.
    
    Args:
        notebook_name: Name of the failed notebook
        error_message: Error details
        hour: Scheduled hour
    """
    try:
        timestamp = get_cairo_time().strftime('%Y-%m-%d %H:%M:%S')
        
        slack_message = f"""❌ *Scheduler Alert - Notebook Failed*

📅 Time: {timestamp} (Cairo)
⏰ Scheduled Hour: {hour}:00
📓 Notebook: `{notebook_name}`

⚠️ *Error:*
```
{error_message[:500]}
```

Please check the scheduler logs for more details."""
        
        send_text_slack(SLACK_CHANNEL, slack_message)
        log_message(f"Slack alert sent for {notebook_name} failure", also_print=True)
    except Exception as e:
        log_message(f"WARNING: Failed to send Slack alert: {e}", also_print=True)


def send_slack_summary(hour, results):
    """
    Send a Slack summary after scheduled tasks complete.
    Only sends if there are any failures.
    
    Args:
        hour: Scheduled hour
        results: Dict of {notebook_name: success_bool}
    """
    failed_notebooks = [nb for nb, success in results.items() if not success]
    
    if not failed_notebooks:
        return  # No failures, no need to send summary
    
    try:
        timestamp = get_cairo_time().strftime('%Y-%m-%d %H:%M:%S')
        total = len(results)
        failed = len(failed_notebooks)
        success = total - failed
        
        status_lines = []
        for notebook, was_success in results.items():
            status = "✅" if was_success else "❌"
            status_lines.append(f"  {status} {notebook}")
        
        slack_message = f"""🔔 *Scheduler Summary - Hour {hour}:00*

📅 Time: {timestamp} (Cairo)
📊 Results: {success}/{total} succeeded, {failed}/{total} failed

*Status:*
{chr(10).join(status_lines)}

⚠️ *Failed Notebooks:* {', '.join(failed_notebooks)}

Please investigate the failures and check logs."""
        
        send_text_slack(SLACK_CHANNEL, slack_message)
        log_message(f"Slack summary sent for hour {hour}", also_print=True)
    except Exception as e:
        log_message(f"WARNING: Failed to send Slack summary: {e}", also_print=True)


print("✓ Helper functions defined")


✓ Helper functions defined


In [3]:
# =============================================================================
# NOTEBOOK EXECUTION FUNCTION
# =============================================================================

def run_notebook(notebook_name, test_mode=False, hour=None):
    """
    Run a Jupyter notebook using nbconvert.
    Returns tuple of (success: bool, error_message: str or None).
    
    Args:
        notebook_name: Key from NOTEBOOKS dict
        test_mode: If True, only log what would happen without executing
        hour: Scheduled hour (for Slack alerts)
    """
    notebook_path = NOTEBOOKS.get(notebook_name)
    
    if not notebook_path:
        error_msg = f"Unknown notebook: {notebook_name}"
        log_message(f"ERROR: {error_msg}")
        return False, error_msg
    
    if not os.path.exists(notebook_path):
        error_msg = f"Notebook not found: {notebook_path}"
        log_message(f"ERROR: {error_msg}")
        return False, error_msg
    
    if test_mode:
        log_message(f"TEST MODE: Would run {notebook_name} ({notebook_path})")
        return True, None
    
    log_message(f"Starting: {notebook_name}")
    
    try:
        # Run notebook using jupyter nbconvert --execute
        # This executes the notebook in place
        cmd = [
            'jupyter', 'nbconvert',
            '--to', 'notebook',
            '--execute',
            '--inplace',
            '--ExecutePreprocessor.timeout=3600',  # 1 hour timeout
            notebook_path
        ]
        
        result = subprocess.run(
            cmd,
            capture_output=True,
            text=True,
            cwd=os.path.dirname(notebook_path)
        )
        
        if result.returncode == 0:
            log_message(f"SUCCESS: {notebook_name} completed")
            return True, None
        else:
            error_msg = result.stderr[:500] if result.stderr else f"Exit code {result.returncode}"
            log_message(f"ERROR: {notebook_name} failed with code {result.returncode}")
            log_message(f"STDERR: {error_msg}")
            return False, error_msg
            
    except Exception as e:
        error_msg = str(e)
        log_message(f"ERROR: Exception running {notebook_name}: {error_msg}")
        return False, error_msg


def run_scheduled_tasks(hour, test_mode=False):
    """
    Run all notebooks scheduled for a given hour.
    Sends Slack alerts if any notebooks fail.
    """
    notebooks = get_scheduled_notebooks(hour)
    
    if not notebooks:
        log_message(f"No notebooks scheduled for hour {hour}")
        return {}
    
    log_message(f"{'='*60}")
    log_message(f"Running scheduled tasks for hour {hour}:00")
    log_message(f"Notebooks: {', '.join(notebooks)}")
    log_message(f"{'='*60}")
    
    results = {}
    errors = {}
    
    for notebook in notebooks:
        success, error_msg = run_notebook(notebook, test_mode=test_mode, hour=hour)
        results[notebook] = success
        
        # Send immediate Slack alert on failure (not in test mode)
        if not success and not test_mode:
            errors[notebook] = error_msg
            send_slack_alert(notebook, error_msg or "Unknown error", hour)
        
        # Small delay between notebooks
        if not test_mode and len(notebooks) > 1:
            time.sleep(5)
    
    # Summary
    log_message(f"\n{'='*60}")
    log_message(f"SUMMARY for hour {hour}:00")
    for notebook, success in results.items():
        status = "✓ SUCCESS" if success else "✗ FAILED"
        log_message(f"  {notebook}: {status}")
    log_message(f"{'='*60}\n")
    
    # Send Slack summary if there were any failures (not in test mode)
    if not test_mode:
        send_slack_summary(hour, results)
    
    return results


print("✓ Notebook execution functions defined")


✓ Notebook execution functions defined


In [4]:
# =============================================================================
# MAIN SCHEDULER FUNCTIONS
# =============================================================================

def run_continuous():
    """
    Run the scheduler continuously, checking every 5 minutes.
    Will execute notebooks at the start of each scheduled hour.
    
    Press Interrupt Kernel (or Ctrl+C in terminal) to stop.
    """
    log_message("Scheduler started in continuous mode")
    print_schedule()
    
    last_run_hour = None
    
    while True:
        try:
            now = get_cairo_time()
            current_hour = now.hour
            current_minute = now.minute
            
            # Run at the beginning of each scheduled hour (within first 10 minutes)
            # and only if we haven't run for this hour yet
            if current_minute <= 10 and current_hour != last_run_hour:
                if current_hour in SCHEDULE:
                    run_scheduled_tasks(current_hour)
                    last_run_hour = current_hour
            
            # Sleep for 9 minutes before checking again
            time.sleep(540)
            
        except KeyboardInterrupt:
            log_message("Scheduler stopped by user (Interrupt Kernel)")
            break
        except Exception as e:
            log_message(f"ERROR in main loop: {str(e)}")
            time.sleep(60)  # Wait a minute before retrying


def run_once():
    """
    Run once for the current hour and exit.
    Useful for manual execution or testing.
    """
    now = get_cairo_time()
    current_hour = now.hour
    
    log_message(f"Running once for current hour ({current_hour}:00)")
    
    if current_hour in SCHEDULE:
        results = run_scheduled_tasks(current_hour)
        return results
    else:
        log_message(f"No notebooks scheduled for hour {current_hour}")
        return {}


def run_test():
    """
    Test mode - show what would run without executing.
    """
    print_schedule()
    
    now = get_cairo_time()
    current_hour = now.hour
    
    print(f"\nTEST MODE - Current hour: {current_hour}:00")
    print("-"*40)
    
    if current_hour in SCHEDULE:
        run_scheduled_tasks(current_hour, test_mode=True)
    else:
        print(f"No notebooks scheduled for hour {current_hour}")


def run_specific_hour(hour, test_mode=False):
    """
    Run notebooks for a specific hour.
    
    Args:
        hour: Hour in 24h format (0-23)
        test_mode: If True, only show what would run
    """
    log_message(f"Running tasks for hour {hour}:00 {'(TEST MODE)' if test_mode else ''}")
    
    if hour in SCHEDULE:
        results = run_scheduled_tasks(hour, test_mode=test_mode)
        return results
    else:
        log_message(f"No notebooks scheduled for hour {hour}")
        return {}


print("✓ Main scheduler functions defined")
print("\nAvailable functions:")
print("  - run_continuous()     : Start the continuous scheduler loop")
print("  - run_once()           : Run current hour's tasks once")
print("  - run_test()           : Test mode - see what would run")
print("  - run_specific_hour(h) : Run tasks for a specific hour")
print("  - print_schedule()     : Print the full schedule")


✓ Main scheduler functions defined

Available functions:
  - run_continuous()     : Start the continuous scheduler loop
  - run_once()           : Run current hour's tasks once
  - run_test()           : Test mode - see what would run
  - run_specific_hour(h) : Run tasks for a specific hour
  - print_schedule()     : Print the full schedule


---
## Print Schedule
Run this cell to see the full schedule:


In [5]:
# =============================================================================
# PRINT SCHEDULE
# =============================================================================
#print_schedule()


---
## Test Mode
Run this cell to see what would run for the current hour (without actually executing):


In [6]:
# =============================================================================
# TEST MODE - See what would run without executing
# =============================================================================
#run_test()


---
## Run Once (Current Hour)
Run this cell to execute only the current hour's scheduled notebooks:


In [7]:
# =============================================================================
# RUN ONCE - Execute current hour's tasks
# =============================================================================
#results = run_once()
#print(f"\nResults: {results}")


---
## Run Specific Hour
Change the hour value below and run to execute a specific hour's tasks:


In [ ]:
# =============================================================================
# RUN SPECIFIC HOUR
# =============================================================================
# Change the hour below (0-23, Cairo time)
HOUR_TO_RUN = 22  # 8 AM

# Set to True to test without executing, False to actually run
TEST_MODE = True


results = run_specific_hour(HOUR_TO_RUN, test_mode=TEST_MODE)
#print(f"\nResults: {results}")


---
## Run Continuous Scheduler
⚠️ **Warning**: This cell runs indefinitely until interrupted!

Run this cell to start the continuous scheduler. It will:
- Check every 5 minutes
- Execute notebooks at the start of each scheduled hour
- Log all activity to `scheduler.log`

To stop: Use **Kernel → Interrupt** or press the stop button.


In [ ]:
# =============================================================================
# RUN CONTINUOUS SCHEDULER
# =============================================================================
# ⚠️ WARNING: This runs indefinitely until interrupted!
# Use Kernel → Interrupt (or the stop button) to stop.
run_continuous()


[2026-02-10 12:48:04] Scheduler started in continuous mode

PRICING LOGIC SCHEDULER - SCHEDULE
Hour       Notebooks                                         
------------------------------------------------------------
08:00      data_extraction, module_2                         
09:00      module_4, treasure_hunt                           
10:00      module_4                                          
11:00      module_4                                          
12:00      module_3                                          
13:00      module_4, wholeSale                               
14:00      module_4                                          
15:00      module_4                                          
16:00      module_4                                          
17:00      module_3                                          
18:00      module_4, wholeSale                               
19:00      module_4                                          
20:00      module_4                   

# Cell kept for future use
